In [ ]:
import json
import pandas as pd 

with open('../input_data/data_presidentielle_2022_nspp.json', 'r', encoding="utf-8") as file:
    liste_sondages_nspp = json.load(file)

In [ ]:
first_tour_scores = []

for index in range(len(liste_sondages_nspp)):
    sondage = liste_sondages_nspp[index]
    nom_institut = sondage["nom_institut"]
    debut_enquete = sondage["debut_enquete"]
    fin_enquete = sondage["fin_enquete"]
    echantillon = sondage["echantillon"]
    tours = sondage["tours"]
    for tour in tours:
        if tour['tour'] == 'Premier tour': #we restrict ourselves to the first round
            # print(tour)
            for hypothesis in tour['hypotheses']:
                    hypo_name = hypothesis['hypothese']
                    scores = []
                    for candidat in hypothesis['candidats']:
                        candidate_info = {
                            'candidat': candidat['candidat'],
                            'parti': candidat['parti'],
                            'intentions': candidat['intentions']
                        }
                        scores.append(candidate_info)
                    first_tour_scores.append({'nom_institut':nom_institut, 'debut_enquete':debut_enquete, 'fin_enquete':fin_enquete, 'echantillon':echantillon, 'hypothese': hypo_name, 'scores': scores})

first_tour_scores

# 1st version: with candidates' names


In [ ]:
unique_candidates = set()
for tour in first_tour_scores:
    for candidate in tour['scores']:
        unique_candidates.add(candidate["candidat"])
unique_candidates

In [ ]:

rows_candidates = []
for tour in first_tour_scores:
    row = {'Company':tour['nom_institut'], 'collectPeriodFrom':tour['debut_enquete'], 'collectPeriodTo':tour['fin_enquete'], 'n':tour['echantillon'], 'Hypothese': tour['hypothese']}
    for candidate in unique_candidates:
        row[candidate] = None
    for candidate in tour['scores']:
        #print(candidate, candidate["candidat"])
        if candidate["candidat"] in unique_candidates:
            row[candidate["candidat"]] = candidate['intentions']
    rows_candidates.append(row)

rows_candidates

#TODO regrouper candidats par parti 

In [ ]:
df_candidates = pd.DataFrame(rows_candidates)
df_candidates['Total'] = df_candidates[list(unique_candidates)].sum(axis=1)
print("we removed", len(df_candidates[df_candidates['Total']!=100]), "outliers polls")
df_candidates = df_candidates[df_candidates['Total']==100] 
df_candidates

In [ ]:
map_people_party = {}

for tour in first_tour_scores:
    for candidate in tour['scores']:
        #print(candidate, candidate["candidat"])
        if len(candidate["parti"])==1:
            map_people_party[candidate["candidat"]] = candidate["parti"][0]
        else: 
            if candidate["parti"] == ['LRM', 'Agir', 'Modem']:
                map_people_party[candidate["candidat"]] = 'LRM'
            elif candidate["parti"] == ['Les Républicains', 'UDI']:
                map_people_party[candidate["candidat"]] = 'Les Républicains'
            # else:
            #     print(candidate, candidate["candidat"])

            


map_people_party['Arnaud Montebourg'] = 'Parti socialiste'
map_people_party['Christiane Taubira'] = 'Parti socialiste'
map_people_party['Jean-Luc Mélenchon'] = 'France insoumise'

map_people_party


In [ ]:
df_candidates_party = df_candidates.iloc[:, :5]
df_candidates = df_candidates.fillna(0)


for name, party in map_people_party.items():
    if party not in df_candidates_party.columns:
        df_candidates_party[party] = 0
    df_candidates_party[party] += df_candidates[name]

df_candidates_party['Total'] = df_candidates_party.iloc[:, 5:].sum(axis=1)

assert len(df_candidates_party[df_candidates_party['Total']!=100]) == 0
df_candidates_party

In [ ]:
desired_columns = [
    'Company',
    'collectPeriodFrom',
    'collectPeriodTo',
    'n',
    'Hypothese',
    'Rassemblement national', 
    'Reconquête', 
    'Debout la France',  #should we keep that one?
    'Les Républicains', 
    'LRM', 
    'EE-LV', 
    'Parti socialiste', 
    'France insoumise'
]

df_candidates_party_small = df_candidates_party[desired_columns].copy()
df_candidates_party_small['Total'] = df_candidates_party_small.loc[:, desired_columns[5:]].sum(axis=1)
df_candidates_party_small


In [ ]:
df_candidates.to_csv('../output_data/df_candidates.csv', index=False, encoding='utf-8')

# 2nd version: with party's names

In [ ]:
unique_parties = set()
for tour in first_tour_scores:
    for candidate in tour['scores']:
        for party in candidate['parti']:
            if party != '':
                unique_parties.add(party)
unique_parties

In [ ]:
link_candidate_party = {'Arnaud Montebourg': 'Montebourg', 'Jean Lassalle':'Résistons', 'Eric Zemmour':'Reconquête', 'Christiane Taubira':'Taubira', 'Jean-Christophe Lagarde':'UDI', 'Jean-Luc Mélenchon':'France insoumise', 'Anne Hidalgo':'Parti socialiste', 'Yannick Jadot':'EE-LV', 'Valérie Pécresse':'Les Républicains', 'Xavier Bertrand':'Les Républicains', 'Bruno Retailleau':'Les Républicains'} #"small" candidates like Montebourg or Taubira are assigned their own party
for key,value in link_candidate_party.items():
    unique_parties.add(value)
unique_parties


# but issue when several candidates from a same party (Montebourg, Taubira and Hidalgo for the Parti socialiste TODO) choix qui peut etre fait : les regrouper ensemble
# in fact is this really an issue, as long as all candidates have a party? need to think more about it 

In [ ]:

rows_parties = []
for tour in first_tour_scores:
    row = {'Company':tour['nom_institut'], 'collectPeriodFrom':tour['debut_enquete'], 'collectPeriodTo':tour['fin_enquete'], 'n':tour['echantillon'], 'Hypothese': tour['hypothese']}
    for party in unique_parties:
        row[party] = 0
    for candidate in tour['scores']:
        if candidate["parti"] == [''] or len(candidate["parti"])>1: #no party or several
            if candidate["candidat"] not in link_candidate_party.keys(): #parties we will need to add by hand
                print(candidate, candidate["candidat"])
                print("no party")
            candidate["parti"] = [link_candidate_party[candidate["candidat"]]]
        for party in candidate['parti']: 
            if party in unique_parties:
                row[party] += candidate['intentions']
    rows_parties.append(row)

#rows_parties

In [ ]:
df_parties = pd.DataFrame(rows_parties)
df_parties['Total'] = df_parties[list(unique_parties)].sum(axis=1)
print(len(df_parties[df_parties['Total']!=100]))
df_parties

In [ ]:
df_parties.to_csv('../output_data/df_parties.csv', index=False, encoding='utf-8')